In [ ]:
%load_ext autoreload
%autoreload 2

%connect_info

In [ ]:
import django
from django.db import transaction

django.setup()

In [ ]:
CONFIG_SHEET_NAME = "CCBP Configuration_v4"

In [ ]:
from ib_iam.models import Project
Project.objects.create(project_id="JGC_DRIVE", display_name)

### Populate project and project roles

In [ ]:
@transaction.atomic()
def populate_project_and_roles():
    from ib_tasks.populate.population_utils import populate_project_roles
    populate_project_roles(CONFIG_SHEET_NAME)
    

@transaction.atomic()
def populate_task_and_boards_related_data():
    
    configuration_sheets = [
        CONFIG_SHEET_NAME
    ]

    for configuration_sheet in configuration_sheets:

        from ib_tasks.populate.population_utils import populate_data
        populate_data(configuration_sheet)
        
        from ib_boards.populate.get_data_from_sheet_for_boards_and_columns import GetBoardsAndColumnsDataFromSheet
        loader = GetBoardsAndColumnsDataFromSheet()
        loader.get_data_from_sheet(configuration_sheet)
        loader.create_boards_and_columns(configuration_sheet)

        
@transaction.atomic()
def populate_project_related_data():

    from ib_tasks.populate.population_utils import populate_projects_for_task_templates
    populate_projects_for_task_templates(CONFIG_SHEET_NAME)
    
    from ib_boards.populate.get_data_from_sheet_for_project_boards import GetSheetDataForProjectBoards
    loader = GetSheetDataForProjectBoards()
    loader.get_data_from_project_boards_sub_sheet(CONFIG_SHEET_NAME)

@transaction.atomic()
def create_indices_in_elasticsearch():
    from elasticsearch_dsl import connections
    from django.conf import settings
    es = connections.create_connection(
        hosts=[settings.ELASTICSEARCH_ENDPOINT], timeout=20
    )

    from ib_tasks.documents.elastic_task import TASK_INDEX_NAME
    es.indices.create(index=TASK_INDEX_NAME)



    
    
@transaction.atomic()
def load_task_template_from_sheet():
#     create_indices_in_elasticsearch()÷
        
    populate_project_and_roles()
    populate_task_and_boards_related_data()
    populate_project_related_data()


In [ ]:
load_task_template_from_sheet()